# Basic Extract Transform and Load(ETL) pipeline using spark and postgres
### Data is scrapped from Stack overflow into the following tables:
- Questions
- Answers
- Users

#### This script is divided into 4 steps to illustrate the process


### Step 1: Data Extraction

In [1]:
import pyspark

from pyspark.sql.types import BooleanType
from pyspark.sql.functions import udf
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql import Window

In [2]:
#initializing spark a spark session
spark = ( 
    SparkSession.builder
                .appName("Stack Overflow Data Wrangling")
                .config("spark.jars", "../jars/postgresql-42.2.8.jar")
                .getOrCreate()
)

In [3]:
#loading the data in dataframes
answers = spark.read.csv("../data/stackoverflow/answers.csv", header=True, multiLine=True, inferSchema=True)
questions = spark.read.csv("../data/stackoverflow/questions.csv", header=True, inferSchema=True,multiLine=True)
users = spark.read.csv("../data/stackoverflow/users.csv", header=True, inferSchema=True,multiLine=True)

In [5]:
answers.columns

['id',
 'user_id',
 'question_id',
 'body',
 'score',
 'comment_count',
 'created_at']

In [6]:
users.columns

['id',
 'display_name',
 'reputation',
 'website_url',
 'location',
 'about_me',
 'views',
 'up_votes',
 'down_votes',
 'image_url',
 'created_at',
 'updated_at']

In [7]:
questions.columns

['id',
 'user_id',
 'title',
 'body',
 'accepted_answer_id',
 'score',
 'view_count',
 'comment_count',
 'created_at']

In [8]:
#transforming the id column names as well as the timestamps too to avoid the ambiguity
answers = answers.withColumnRenamed('id', 'answer_id').withColumnRenamed('created_at', 'answer_created_at').withColumnRenamed('body','answer_body').withColumnRenamed('score','answer_score').withColumnRenamed('comment_count','answer_comment_count')
questions = questions.withColumnRenamed('id', 'question_id').withColumnRenamed('created_at', 'question_created_at').withColumnRenamed('body','question_body').withColumnRenamed('score','question_score')
users = users.withColumnRenamed('id', 'user_id').withColumnRenamed('created_at', 'user_created_at').withColumnRenamed('updated_at', 'user_updated_at')

#### Step 2: Data Transformation

In [9]:
# getting only indian users..
india_users = users.filter(users.location.contains('India'))

In [10]:
india_users.select("location").show()

+--------------------+
|            location|
+--------------------+
|Bangalore, Karnat...|
|New Delhi, Delhi,...|
|Gharaunda, Haryan...|
|    New Delhi, India|
|Jalandhar, Punjab...|
|Surat, Gujarat, I...|
|Darjeeling, West ...|
|Pune, Maharashtra...|
|Mumbai, Maharasht...|
|Bangalore, Karnat...|
|Mumbai, Maharasht...|
|Bangalore, Karnat...|
|Mumbai, Maharasht...|
|Hyderabad, Telang...|
|Indore, Madhya Pr...|
|               India|
|Bangalore, Karnat...|
|Naya Raipur, Chha...|
|Bangalore, Karnat...|
|Chennai, Tamil Na...|
+--------------------+
only showing top 20 rows



In [11]:
#extracting the country and city into new columns
cols = F.split(india_users['location'], ',' )
india_users = india_users.withColumn('city', cols.getItem(0))
india_users = india_users.withColumn('country', cols.getItem(2))

In [12]:
# rows where city was not quoted the country are being taken as None
india_users.select(['location','city','country']).show()

+--------------------+-----------+-------+
|            location|       city|country|
+--------------------+-----------+-------+
|Bangalore, Karnat...|  Bangalore|  India|
|New Delhi, Delhi,...|  New Delhi|  India|
|Gharaunda, Haryan...|  Gharaunda|  India|
|    New Delhi, India|  New Delhi|   null|
|Jalandhar, Punjab...|  Jalandhar|  India|
|Surat, Gujarat, I...|      Surat|  India|
|Darjeeling, West ...| Darjeeling|  India|
|Pune, Maharashtra...|       Pune|  India|
|Mumbai, Maharasht...|     Mumbai|  India|
|Bangalore, Karnat...|  Bangalore|  India|
|Mumbai, Maharasht...|     Mumbai|  India|
|Bangalore, Karnat...|  Bangalore|  India|
|Mumbai, Maharasht...|     Mumbai|  India|
|Hyderabad, Telang...|  Hyderabad|  India|
|Indore, Madhya Pr...|     Indore|  India|
|               India|      India|   null|
|Bangalore, Karnat...|  Bangalore|  India|
|Naya Raipur, Chha...|Naya Raipur|  India|
|Bangalore, Karnat...|  Bangalore|  India|
|Chennai, Tamil Na...|    Chennai|  India|
+----------

In [13]:
# an inner join of the filtered users df with the question df
df = india_users.join(questions, on='user_id', how='left')
df.columns

['user_id',
 'display_name',
 'reputation',
 'website_url',
 'location',
 'about_me',
 'views',
 'up_votes',
 'down_votes',
 'image_url',
 'user_created_at',
 'user_updated_at',
 'city',
 'country',
 'question_id',
 'title',
 'question_body',
 'accepted_answer_id',
 'question_score',
 'view_count',
 'comment_count',
 'question_created_at']

In [14]:
# selecting only questions with at least 20 view counts
df = df.filter(df['view_count'] >= 20)

In [15]:
# joining the resultant table to the users answers table
df = df.join(answers, on=['question_id','user_id'], how='left')
df.columns

['question_id',
 'user_id',
 'display_name',
 'reputation',
 'website_url',
 'location',
 'about_me',
 'views',
 'up_votes',
 'down_votes',
 'image_url',
 'user_created_at',
 'user_updated_at',
 'city',
 'country',
 'title',
 'question_body',
 'accepted_answer_id',
 'question_score',
 'view_count',
 'comment_count',
 'question_created_at',
 'answer_id',
 'answer_body',
 'answer_score',
 'answer_comment_count',
 'answer_created_at']

In [16]:
df.count()

130

### Step 3: Data Loading

In [20]:
# writing to the a postgres database
# assuming the schema is already created.
df.write.format("jdbc").options(
    url='jdbc:postgresql://localhost:5432/postgres',
    driver='org.postgresql.Driver',
    user='postgres',
    password='postgres',
    dbtable='stackoverflow_filtered.results'
).save(mode='append')

### Differences bewteen a view and a materialised view.
- A view is never stored it is only displayed a materialised view is stored on the disk
- A view is a virtual table but a materialized view is a physical copy of a table
- A view is updated anytime is used but a materialised view has to be updated manually